# Affinity Analysis

**Affinity Analysis (or Market Basket Analysis)** finds patterns to determine connections between purchases in order that stores can increase their cross-selling potential.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data
data_path = 'Online_Retail.xlsx'

data = pd.read_excel(data_path)

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Data Wrangling

In [4]:
# Remove additional spaces from Description
data['Description'] = data['Description'].str.strip()
data.shape

(541909, 8)

In [5]:
# Drop rows with Nan values
data = data.dropna()
data.shape

(406829, 8)

In [6]:
# Remove cancelled or returned orders
data = data[data['Quantity']>0]
data.shape

(397924, 8)

In [7]:
# Data type conversion
data['InvoiceNo'] = data['InvoiceNo'].astype(str)
data['StockCode'] = data['StockCode'].astype(str)

In [8]:
# Restrict the analysis to United Kingdom
data_uk = data[data['Country']=='United Kingdom']
data_uk.shape

(354345, 8)

## Market Basket Analysis

**Note:** Include products that appear in more tham 3% of transactions as it would help eliminate the scarcely purchased products and reduce the product count which in turn reduces the computation time as there is an exponential increase in computation with a number of items.

In [9]:
total_transactions = data_uk['InvoiceNo'].nunique()
print(f'Total Transactions: {total_transactions}')
transac_3perc = 0.03*total_transactions
print(f"3% of Total Transactions: {round(transac_3perc)}")

Total Transactions: 16649
3% of Total Transactions: 499


In [10]:
group_data = (data_uk.groupby('StockCode')['InvoiceNo']
              .nunique()
              .to_frame()
              .reset_index()
              .rename(columns={'InvoiceNo':'NoOfTransactions'}))
group_data

,StockCode,NoOfTransactions
0,10002,36
1,10080,21
2,10120,28
3,10123C,3
4,10124A,5
...,...,...
3640,C2,30
3641,DOT,16
3642,M,227
3643,PADS,4


In [11]:
product_include = group_data[group_data['NoOfTransactions']>transac_3perc]['StockCode'].tolist()
print(len(product_include))

92


In [12]:
data_uk = data_uk[data_uk['StockCode'].isin(product_include)]
data_uk.shape

(68772, 8)

### Create a dictionary to map StockCode with Description

In [13]:
# Create a dictionary of item code and item description
item_dict = data_uk.groupby(['StockCode'])['Description'].agg(pd.Series.mode).to_dict()

### List the products included in each transaction

In [14]:
transaction_data = data_uk.groupby('InvoiceNo')['StockCode'].unique().to_dict()

### Create a basket list: List of all products per transaction

In [15]:
basket_list = list(transaction_data.values())

### Since Affinity analysis builds the association between 2 or more items that is bought according to historical data, it would less useless to have a transaction with a single item.

In [16]:
basket_list = [val for val in basket_list if len(val)>1]
len(basket_list)

10723

### Create a list of all possible combinations of size 2 from available items

In [17]:
item_list = list(item_dict.keys())

In [18]:
item_comb = list(combinations(item_list, 2))
reverse_comb_list = [val[::-1] for val in item_comb]
item_comb = item_comb + reverse_comb_list
print(f"Total Combinations of items: {len(item_comb)}")

Total Combinations of items: 8372


### Create a DataFrame of Antecedents and Consequents

- **Antecedents:** These are the items that are bought before a consequent.
- **Consequents:** These are the items that are bought after a antecedent.

In Affinity analysis we try to build an association between an Antecedent and a Consequents to see how the purchase of an item X affects the purchase of an item Y after X has been put in the basket.

In [19]:
affinity_data = pd.DataFrame(item_comb, columns=['Antecedent', 'Consequent'])
affinity_data.head()

,Antecedent,Consequent
0,20685,20724
1,20685,20725
2,20685,20726
3,20685,20727
4,20685,20728


### Support

Support is how frequently an item or a combination of items occur in the market basket.

In [20]:
def get_support(items):
    if not isinstance(items, list):
        items = [items]
    support_val = sum(set(items).issubset(set(i)) for i in basket_list)/len(basket_list)
    return support_val

In [21]:
affinity_data['Antecedent_support'] = affinity_data['Antecedent'].map(get_support)

In [22]:
affinity_data['Consequent_support'] = affinity_data['Consequent'].map(get_support)

In [23]:
affinity_data['support'] = affinity_data.apply(lambda row: get_support([row['Antecedent'],row['Consequent']]), axis=1)

In [24]:
affinity_data.head()

,Antecedent,Consequent,Antecedent_support,Consequent_support,support
0,20685,20724,0.043271,0.056048,0.002984
1,20685,20725,0.043271,0.103796,0.006155
2,20685,20726,0.043271,0.062296,0.003171
3,20685,20727,0.043271,0.091672,0.004476
4,20685,20728,0.043271,0.081787,0.004663


### Confidence

It is the percentage of cases in which a consequent appears given that the antecedent has occurred.

$P(consequent|antecedent)$

In [25]:
affinity_data['confidence'] = affinity_data['support']/affinity_data['Antecedent_support']

In [26]:
affinity_data.head()

,Antecedent,Consequent,Antecedent_support,Consequent_support,support,confidence
0,20685,20724,0.043271,0.056048,0.002984,0.068966
1,20685,20725,0.043271,0.103796,0.006155,0.142241
2,20685,20726,0.043271,0.062296,0.003171,0.073276
3,20685,20727,0.043271,0.091672,0.004476,0.103448
4,20685,20728,0.043271,0.081787,0.004663,0.107759


### Lift

It is a factor by which the likelihood of consequent increases given an antecedent.

Lift is the ratio of the likelihood of finding Consequent in a basket known to contain Antecedent, to the likelihood of finding Consequent in any random basket.

$\frac{P(consequent|antecedent)}{P(consequent)}$

In [27]:
affinity_data['lift'] = affinity_data['confidence']/affinity_data['Consequent_support']

In [28]:
affinity_data.head()

,Antecedent,Consequent,Antecedent_support,Consequent_support,support,confidence,lift
0,20685,20724,0.043271,0.056048,0.002984,0.068966,1.230478
1,20685,20725,0.043271,0.103796,0.006155,0.142241,1.370399
2,20685,20726,0.043271,0.062296,0.003171,0.073276,1.176253
3,20685,20727,0.043271,0.091672,0.004476,0.103448,1.128460
4,20685,20728,0.043271,0.081787,0.004663,0.107759,1.317555


### Conviction

It compares the probability that X appears without Y if they were dependent with the actual frequency of the appearance of X without Y.

In [29]:
def get_conviction(confidence, consequent_support):
    if confidence==1:
        return 0
    else:
        return (1-consequent_support)/(1-confidence)

In [30]:
affinity_data['conviction'] = affinity_data.apply(lambda row: get_conviction(row['confidence'], row['Consequent_support']), 
                                                  axis=1)

In [31]:
affinity_data.head()

,Antecedent,Consequent,Antecedent_support,Consequent_support,support,confidence,lift,conviction
0,20685,20724,0.043271,0.056048,0.002984,0.068966,1.230478,1.013875
1,20685,20725,0.043271,0.103796,0.006155,0.142241,1.370399,1.044821
2,20685,20726,0.043271,0.062296,0.003171,0.073276,1.176253,1.011848
3,20685,20727,0.043271,0.091672,0.004476,0.103448,1.128460,1.013135
4,20685,20728,0.043271,0.081787,0.004663,0.107759,1.317555,1.029108


### Leverage

It is to measure how much more counting is obtained from the co-occurrence of the antecedent and consequent from the expected, i.e., from independence.

In [32]:
affinity_data['leverage'] = affinity_data['support'] - (affinity_data['Antecedent_support']*affinity_data['Consequent_support'])

In [33]:
affinity_data.head()

,Antecedent,Consequent,Antecedent_support,Consequent_support,support,confidence,lift,conviction,leverage
0,20685,20724,0.043271,0.056048,0.002984,0.068966,1.230478,1.013875,0.000559
1,20685,20725,0.043271,0.103796,0.006155,0.142241,1.370399,1.044821,0.001664
2,20685,20726,0.043271,0.062296,0.003171,0.073276,1.176253,1.011848,0.000475
3,20685,20727,0.043271,0.091672,0.004476,0.103448,1.128460,1.013135,0.000510
4,20685,20728,0.043271,0.081787,0.004663,0.107759,1.317555,1.029108,0.001124


In [34]:
affinity_data = affinity_data[affinity_data['support']>=0.03]

In [35]:
affinity_data['Antecedent'] = affinity_data['Antecedent'].map(item_dict)

In [36]:
affinity_data['Consequent'] = affinity_data['Consequent'].map(item_dict)

In [37]:
affinity_data.head()

,Antecedent,Consequent,Antecedent_support,Consequent_support,support,confidence,lift,conviction,leverage
181,LUNCH BAG RED RETROSPOT,LUNCH BAG WOODLAND,0.103796,0.062296,0.033946,0.327044,5.249840,1.393411,0.027480
182,LUNCH BAG RED RETROSPOT,LUNCH BAG BLACK SKULL.,0.103796,0.091672,0.045137,0.434861,4.743654,1.607264,0.035621
183,LUNCH BAG RED RETROSPOT,LUNCH BAG CARS BLUE,0.103796,0.081787,0.038515,0.371069,4.537029,1.459959,0.030026
210,LUNCH BAG RED RETROSPOT,LUNCH BAG SPACEBOY DESIGN,0.103796,0.080854,0.038236,0.368374,4.556023,1.455205,0.029843
211,LUNCH BAG RED RETROSPOT,LUNCH BAG SUKI DESIGN,0.103796,0.088968,0.043458,0.418688,4.706073,1.567201,0.034224


In [38]:
affinity_data.sort_values('lift', ascending=False).head()

,Antecedent,Consequent,Antecedent_support,Consequent_support,support,confidence,lift,conviction,leverage
3151,GREEN REGENCY TEACUP AND SAUCER,ROSES REGENCY TEACUP AND SAUCER,0.056234,0.061830,0.044391,0.789386,12.767105,4.454462,0.040914
7337,ROSES REGENCY TEACUP AND SAUCER,GREEN REGENCY TEACUP AND SAUCER,0.061830,0.056234,0.044391,0.717949,12.767105,3.346079,0.040914
8119,GARDENERS KNEELING PAD KEEP CALM,GARDENERS KNEELING PAD CUP OF TEA,0.068638,0.057820,0.042712,0.622283,10.762478,2.494406,0.038743
3933,GARDENERS KNEELING PAD CUP OF TEA,GARDENERS KNEELING PAD KEEP CALM,0.057820,0.068638,0.042712,0.738710,10.762478,3.564474,0.038743
3325,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE RED,0.062296,0.068824,0.042339,0.679641,9.875051,2.906661,0.038051
